<h1>Diffie-Hellman key exchange </h1>
<h2>Introduction</h2>
<p>
 From Wikipedia, the free encyclopedia<br>
Diffie–Hellman key exchange is a method of securely exchanging cryptographic keys over a public channel and was one of the first public-key protocols as conceived by Ralph Merkle and named after Whitfield Diffie and Martin Hellman. DH is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. and it's based on the Discrete Logarithm Problem.
</p>

<h2>The Algorithm</h2>
If Alice and Bob want to exchange some keys  so they have follow these steps :
 <ul>
     <li> Choose a big prime number p and choose a generator  g of the group $\mathbb{Z}/p\mathbb{Z} $
     <li>   Alice pick a random number a (his private and Bob pick another one b $(2<a,b<p-1$)
     <li> Alice Calculates her Public Key A ($ g^{a} \equiv A [p] $) and Bob  Calculate her Public Key B ($ g^{b} \equiv B[p] $)
       <li>  Exchange the Public Key between them and Calculate the Shared Secret : $  SS \equiv A^{b} \equiv (g^{a})^{b} \equiv (g^{b})^{a} \equiv B^{a} [p] $
       <li> Using the AES and Shared Secret they can decrypt and encrypt any data they want.

In [3]:
from Crypto.Util.number import getPrime,long_to_bytes,bytes_to_long
from Crypto.Random.random import randint


p=getPrime(128)
print(f"the prime p is : {p}")
#generaly 2 is the most known generator for many Z/pZ groups
g=2

def getKey():
    n = randint(2,p-1)
    return n , pow(g,n,p)



na,A= getKey()
##Alice's private key = na
print(f"Alice's Public Key A = {A}")
nb,B=getKey()
##Bob's private key = nb
print(f"Bob's Public Key B = {B}")
assert nb != na

print( pow(A,nb,p)==pow(B,na,p))
#the Shared secret
SS=pow(A,nb,p)
print(f"the shared secret SS = {SS}")

the prime p is : 266471911929868067965211065234204250411
Alice's Public Key A = 46406332798894122601408712341556794169
Bob's Public Key B = 219710608454794974798709120942827766926
True
the shared secret SS = 204702427017377856937045885906147279286


In [4]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
import hashlib
import os


#this is a sample of the encryption using AES-CBC mode 


Message = b'this is a secret message'

def encrypt(shared_secret: int):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(Message, 16))
    # Prepare data to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_message'] = ciphertext.hex()
    return data

encrypted=encrypt(SS)
print("Alice send to Bob :", encrypted)

def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')


decrypted=decrypt(SS,encrypted['iv'],encrypted['encrypted_message'])
print(f" the decrypted message is : {decrypted} ")


Alice send to Bob : {'iv': 'd508b373c673563a952ff449b9297e58', 'encrypted_message': 'c9af3d49b36062fe1a575b5647716af2888e65b871b43fd32733f2da630a15c3'}
 the decrypted message is : this is a secret message 
